In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import plotly.express as px
from tqdm import tqdm
import random

import optuna
from optuna.samplers import RandomSampler
from optuna.samplers import TPESampler

optuna.logging.set_verbosity(optuna.logging.ERROR)

In [2]:
import sys
sys.path.append('../src')

import vrpp
import importlib
importlib.reload(vrpp)

<module 'vrpp' from '/Users/sergak/Documents/Neyro_sets/optimal_encashement/notebooks/../src/vrpp.py'>

In [3]:
dist = pd.read_csv('../data/raw/times v4.csv')
le = preprocessing.LabelEncoder()
le.fit(dist['Origin_tid'])
dist['from_int'] = le.transform(dist['Origin_tid'])
dist['to_int'] = le.transform(dist['Destination_tid'])
dist.head()

,Origin_tid,Destination_tid,Total_Time,from_int,to_int
0,636538,683103,15.32,864,1354
1,636538,634763,16.20,864,624
2,636538,683128,16.27,864,1358
3,636538,683789,16.77,864,1374
4,636538,634709,17.67,864,603


In [4]:
data = pd.read_excel('../data/raw/terminal_data_hackathon v4.xlsx', 'Incomes')
data.head()

,TID,остаток на 31.08.2022 (входящий),2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,2022-09-05 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,...,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
0,406136,160000,90000,105000,99000,107000,110000,60000,75000,89000,...,91000,78000,0,165000,0,189000,106000,94000,75000,74000
1,406139,387000,103000,206000,168000,124000,78000,165000,164000,174000,...,164000,153000,151000,157000,206000,182000,123000,138000,112000,179000
2,406145,287000,143000,136000,124000,117000,123000,140000,139000,138000,...,119000,100000,179000,169000,118000,118000,114000,128000,121000,124000
3,406148,355000,50000,73000,53000,65000,75000,100000,53000,52000,...,48000,55000,65000,85000,95000,68000,62000,0,118000,70000
4,406180,597000,96000,82000,71000,72000,86000,55000,55000,75000,...,82000,56000,70000,59000,105000,70000,77000,87000,59000,55000


In [5]:
config = {'num_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': 100,
          'terminal_service_persent': 0, #0.01
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10,
          'left_days_coef': 0,
          'encashment_coef': 0,}

In [12]:
real_cash = (data[data.columns[1:]].values * (1 + 0.2 * (random.random() - 0.5))).copy()
# real_cash = data[data.columns[1:]].values.copy()
cash = real_cash.copy()

In [13]:
cash = cash.astype(int)
cash.shape

(1630, 92)

In [14]:
INF = 1e9

In [15]:
def get_cost(days_left):
    if days_left == 0: return INF
    return 2 ** (config['max_not_service_days'] - days_left)

In [ ]:
cash = real_cash.copy()
num_vehicles = 5
num_terminals = cash.shape[0]
time_until_force = [config['max_not_service_days'] for i in range(num_terminals)]
cur_cash = cash[:, 0]
cash = cash[:, 1:]

hist = {'visited': [], 'loss': []}
bad = False

myvrp = vrpp.VRPP(dist, 10, 10 * 60, num_vehicles, solution_limit=100, time_limit=100, dead_loss=False)

days = 90
big_id = config['max_not_service_days']
for day in tqdm(range(days)):
    mask = []
    cost = []
    to_counter = [0 for i in range(config['max_not_service_days'] + 1)]
    
    for i in range(num_terminals):
        force = time_until_force[i]
        if cur_cash[i] > config['max_terminal_money']:
            force = 0
            
            
        force_for_show = time_until_force[i]
        current_money = cur_cash[i]
        for forecast in range(min(force_for_show, days - day)):
            if current_money > config['max_terminal_money']:
                force_for_show = forecast
                break
            if not forecast:
                current_money += cash[i][day + forecast]
            else:
                current_money += cash[i][day + forecast] # check later
                
        mask.append(1)
        cost.append(int(get_cost(force)))
        to_counter[force_for_show] += 1
    
    print(to_counter)
    # # for i in range(100):
    # #     print(" " * (4 - len(str(cost[i]))) + str(cost[i]), end=' ')
    # # print()
    # print(" "cost[:100])
    
    visited, paths = myvrp.find_vrp(cost, mask)
    hist['visited'].append(visited)
    
    prev = []
    cur_loss = 0
    for i in range(num_terminals):
        if cur_cash[i] > config['max_terminal_money'] or time_until_force[i] == 0:
            if not visited[i]:
                print('Dead')
                bad = True
                break

        if visited[i]:
            cur_loss += max(config['terminal_service_cost'], cur_cash[i] * config['terminal_service_persent'])
            cur_cash[i] = 0
            time_until_force[i] = config['max_not_service_days']
        else:
            time_until_force[i] -= 1
            
        cur_loss += cur_cash[i] * config['persent_day_income']
        cur_cash[i] += cash[i][day]
    
    if bad:
        break
    
    hist['loss'].append(cur_loss)
    big_id -= 1
    
print(sum(hist['loss']))

  0%|                                                                                                          | 0/90 [00:00<?, ?it/s]

[10, 7, 22, 37, 35, 43, 63, 62, 63, 79, 95, 75, 86, 73, 880]


  1%|█                                                                                                 | 1/90 [00:19<29:25, 19.84s/it]

[6, 18, 30, 34, 38, 63, 60, 62, 74, 87, 70, 84, 77, 752, 175]
